In [5]:
%load_ext autoreload
%autoreload 2

In [8]:
from data_cleaning import *
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sb
from scipy import stats
from scipy.stats import pearsonr
from tqdm import tqdm

In [9]:
def dataset_creator(models_names: list, columns_names: list, k1: int):
    header = pd.MultiIndex.from_product([models_names, columns_names])
    df = pd.DataFrame(columns=header)
    df['KFold'] = np.arange(1, k1+1)
    df.set_index('KFold', inplace=True)
    return df

def twolevelcv(X: np.array, y: np.array, k1: int, k2: int, models: list, params: dict, rs: int):
    """Allows to compute two level crossvalidation.

    Args:
        X (np.array): Features (numeric)
        y (np.array): Class (objective variable)
        k1 (int): Nº of outer folds
        k2 (int): Nº of inner folds
        models (list): List of models for comparison
        params (dict): Dictionary including the set of parameters. In this case we only tune 1 parameter per model.
        rs (int): Random state
    Returns:
        df: Dataframe
    """
    test_error_dict = {}
    k = 0
    names = [type(m).__name__ for m in models]
    col_names = ['Param. Value', 'Error']
    df = dataset_creator(names, col_names, k1)
    kf1 = StratifiedKFold(k1, shuffle = True, random_state=rs)
    # first level split
    for train_idx1, test_idx1 in kf1.split(X, y):
        k += 1
        kf2 = StratifiedKFold(k2, shuffle = True, random_state=rs)
        print(f'Computing KFold {k}/{k1}...')
        # second level split
        for train_idx2, test_idx2 in tqdm(kf2.split(X[train_idx1, :], y[train_idx1]), total = k2):
            X_train = X[train_idx2, :]
            y_train = y[train_idx2]
            X_test = X[test_idx2, :]
            y_test = y[test_idx2]
            for name, model in zip(names, models):
                if name != 'DummyClassifier':
                    pname = list(params[name].keys())[0]
                    error_test = []
                    for p_ in params[name][pname]:
                        pdict = {pname: p_}
                        model = model.set_params(**pdict)
                        # train the model
                        model.fit(X_train, y_train)
                        # evaluate performance
                        pred2_test = model.predict(X_test)
                        error_test.append(np.sum(pred2_test != y_test)/ y_test.shape[0])
                    min_param = params[name][pname][np.argmin(error_test)]
                else:
                    model.fit(X_train, y_train)
                    pred2_test = model.predict(X_test)
                    error_test = np.sum(pred2_test != y_test)/ y_test.shape[0]
                    min_param = np.NaN
                df.loc(axis = 1)[name, 'Error'][k] = np.min(error_test)
                df.loc(axis = 1)[name, 'Param. Value'][k] = min_param
    return df, test_idx1

### **Data flow**

In [10]:
df_raw, df_Xn_raw = load_data()

In [11]:
fill_methods = ['mean', 'median', 'most_frequent']
std_methods = ['standard', 'minmax', 'maxabs', 'robust']

In [12]:
X, y = transform_data(df_raw, fill_methods[0], std_methods[0])

In [14]:
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.dummy import DummyRegressor
from sklearn.neural_network import MLPRegressor

random_state = 3
params = {}
lam = np.logspace(-6, 2, 100)
C = 1/ lam
# C = [200000000, 10000000, 0.1519911082952933, 0.2848035868435805 ]
params['LogisticRegression'] = {'C': C}
params['DummyRegressor'] = {'strategy': ['mean', 'median', 'quantile']}
params['MLPRegressor'] = {'hidden_layer_sizes': [(8, ), (16, ), (20, )]}
params['LinearRegression'] = {'fit_intercept': [True, False]}
params['Ridge'] = {'alpha': [0.1, 1.0, 10.0]}
models = [LogisticRegression(solver='saga', max_iter=1000000, random_state= random_state, tol = 0.003, n_jobs = -1),
          DummyRegressor(),
          MLPRegressor(solver='adam', activation='logistic', alpha=1e-4, random_state=random_state, max_iter=1000,
          early_stopping=True, validation_fraction=0.2, warm_start=True, verbose=False),
          LinearRegression(n_jobs=-1),
          Ridge(random_state=random_state)]
k1 = 10
k2 = 10
Table, test_set_outer = twolevelcv(df_raw,k1=k1,k2=k2,model=models,params=params,r_s=random_state)

NameError: name 'X_stdz' is not defined

In [3]:
import numpy as np

In [5]:
np.sqrt(1.8e04)

134.16407864998737